# Genomics Commons Docker Image: A Simple Example

Docker images are self-contained, lightweight software packages equipped with all the essentials to run a program: the code, runtime, system tools, libraries, and settings. In genomics, Docker images offer invaluable consistency for computational analyses. The Genomics Commons Docker image, in particular, is a curated collection of tools and libraries tailored for genomics research. In this tutorial, we will guide you through a step-by-step process to download and inspect this image. Furthermore, we will demonstrate its use in calculating the identity by descent using PLINK.

## Docker Image Setup

To begin, we'll download the latest version of the Genomics Commons Docker Image from the GitHub Container Registry `ghcr.io`. This ensures that we're working with the most recent version of the tools and software installed within the image.

In [1]:
docker pull ghcr.io/ecrrm/genomics-commons

Using default tag: latest
latest: Pulling from ecrrm/genomics-commons
Digest: sha256:e877a246d8d9b37223d47b97b3bb81fe3058e947502aac553bf4af9047e31cbe
Status: Image is up to date for ghcr.io/ecrrm/genomics-commons:latest
ghcr.io/ecrrm/genomics-commons:latest



Now, let's verify the version of `plink` installed in the Docker image. `plink` is a free, open-source whole genome association analysis toolset, designed to perform a range of basic, large-scale analyses in a computationally efficient manner. Knowing the version helps in ensuring compatibility with specific analysis pipelines.


In [2]:
docker run --rm ghcr.io/ecrrm/genomics-commons plink --version

PLINK v1.90b7 64-bit (16 Jan 2023)


The output shows that the version of `plink` installed in the Docker image is `v1.90b7`, released on `16 Jan 2023`. Having this information can be crucial when ensuring compatibility with certain genomics workflows.

## Data Acquisition

### Description of Samples NA12877 and NA12878
The samples NA12877 and NA12878 are part of [the 1000 Genomes Project](https://www.internationalgenome.org/) and represent two of the high-coverage sequenced individuals.

- **NA12877**: This sample corresponds to a child from a family trio studied as part of the project.
- **NA12878**: This is the mother of NA12877 and is one of the most sequenced genomes in the world. The sample is frequently used as a benchmark in genomics studies and serves as a reference in many projects due to its extensive sequencing and characterization.

We'll start by downloading a VCF (Variant Call Format) file for the sample NA12877. A VCF file is a standard format in genomics that contains information about genetic variations found in specific samples. This format is widely used in genomics research and bioinformatics pipelines.

First, let's create a temporary directory to store our downloaded data. This directory will hold the VCF (Variant Call Format) files, which contain information about genetic variations found in specific samples. VCF is a standard format in genomics for representing such variations.

In [3]:
tmpdir=$(mktemp -d)

echo $tmpdir

/tmp/tmp.1pZR3ymSfT


To ensure that the directory was successfully created, we'll list its contents. This step is a basic verification to confirm everything is set up correctly before we proceed with data acquisition.

In [4]:
ls -al $tmpdir

total 200
drwx------  2 ahmed ahmed   4096 Aug 13 07:48 .
drwxrwxrwt 54 root  root  196608 Aug 13 07:48 ..


Now, let's download the VCF file for sample, NA12878.

In [5]:
curl -o $tmpdir/NA12877.vcf.gz https://s3.eu-central-1.amazonaws.com/platinum-genomes/2017-1.0/hg38/small_variants/NA12877/NA12877.vcf.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 35.2M  100 35.2M    0     0  7581k      0  0:00:04  0:00:04 --:--:-- 8286k


The output confirms that the VCF file for the sample NA12877 was successfully downloaded.

Now, let's download another VCF file for a different sample, NA12878. This will give us multiple samples to work with in subsequent steps.

In [6]:
curl -o $tmpdir/NA12878.vcf.gz https://s3.eu-central-1.amazonaws.com/platinum-genomes/2017-1.0/hg38/small_variants/NA12878/NA12878.vcf.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 35.4M  100 35.4M    0     0  7645k      0  0:00:04  0:00:04 --:--:-- 8975k


The output indicates that the VCF file for the sample NA12878 was successfully downloaded.

## Data Preparation

After downloading the VCFs file for the samples, we'll use the `tabix` command (from within our Docker image) to index these files. Indexing with `tabix` allows for efficient random access to specific regions of the VCF file, making it quicker to retrieve variant information for specific genomic locations.

In [7]:
docker run --rm -v $tmpdir:/data ghcr.io/ecrrm/genomics-commons tabix -f /data/NA12877.vcf.gz

Now, let's list the contents of our temporary directory again. This step helps us verify the presence of both the downloaded VCF file and its index created using `tabix`.

In [8]:
ls -al $tmpdir

total 74076
drwx------  2 ahmed ahmed     4096 Aug 13 07:48 .
drwxrwxrwt 54 root  root    196608 Aug 13 07:48 ..
-rw-r--r--  1 ahmed ahmed 36943942 Aug 13 07:48 NA12877.vcf.gz
-rw-r--r--  1 root  root   1550031 Aug 13 07:48 NA12877.vcf.gz.tbi
-rw-r--r--  1 ahmed ahmed 37138627 Aug 13 07:48 NA12878.vcf.gz


The directory listing confirms the presence of both the VCF file (`NA12877.vcf.gz`) and its index (`NA12877.vcf.gz.tbi`). With these files ready, we can proceed to the data preparation steps.

In [9]:
docker run --rm -v $tmpdir:/data ghcr.io/ecrrm/genomics-commons tabix -f /data/NA12878.vcf.gz

Let's once again list the contents of our temporary directory. This verification step ensures we have both the VCF files and their respective indexes for samples NA12877 and NA12878.

In [10]:
ls -al $tmpdir

total 75596
drwx------  2 ahmed ahmed     4096 Aug 13 07:48 .
drwxrwxrwt 54 root  root    196608 Aug 13 07:48 ..
-rw-r--r--  1 ahmed ahmed 36943942 Aug 13 07:48 NA12877.vcf.gz
-rw-r--r--  1 root  root   1550031 Aug 13 07:48 NA12877.vcf.gz.tbi
-rw-r--r--  1 ahmed ahmed 37138627 Aug 13 07:48 NA12878.vcf.gz
-rw-r--r--  1 root  root   1554018 Aug 13 07:48 NA12878.vcf.gz.tbi


The directory listing confirms the presence of the VCF files and their indexes for both samples: `NA12877` and `NA12878`. With these files in place, we can move forward to the analysis.

After downloading and indexing the VCF files for our samples, we can now merge them into a single VCF file. Merging VCF files is a common operation when working with genomics data from multiple samples. This allows us to perform joint variant calling, which can increase the accuracy of variant detection.

To combine the VCF files for both samples (`NA12877` and `NA12878`), we'll use the `vcf-merge` tool. Merging multiple VCF files allows us to consolidate variant information from different samples into a single file, facilitating joint analyses.

In [11]:
docker run --rm -v $tmpdir:/data ghcr.io/ecrrm/genomics-commons vcf-merge /data/NA12877.vcf.gz /data/NA12878.vcf.gz > $tmpdir/merged.vcf

Using column name 'NA12877' for /data/NA12877.vcf.gz:NA12877
Using column name 'NA12878' for /data/NA12878.vcf.gz:NA12878


Let's now list the contents of our directory to verify the presence of the merged VCF file. This step ensures the merging process was successful and the merged file is ready for further analysis.

In [12]:
ls -al $tmpdir

total 756524
drwx------  2 ahmed ahmed      4096 Aug 13 07:48 .
drwxrwxrwt 54 root  root     196608 Aug 13 07:55 ..
-rw-r--r--  1 ahmed ahmed 697262816 Aug 13 07:55 merged.vcf
-rw-r--r--  1 ahmed ahmed  36943942 Aug 13 07:48 NA12877.vcf.gz
-rw-r--r--  1 root  root    1550031 Aug 13 07:48 NA12877.vcf.gz.tbi
-rw-r--r--  1 ahmed ahmed  37138627 Aug 13 07:48 NA12878.vcf.gz
-rw-r--r--  1 root  root    1554018 Aug 13 07:48 NA12878.vcf.gz.tbi


The merged VCF file contains information about genetic variants found in both samples. The VCF format consists of a header section and a data section. The header contains metadata about the file and the data section lists the genetic variants along with various annotations.

The directory listing shows the `merged.vcf` file alongside the VCF files and indexes for both samples (`NA12877` and `NA12878`). This confirms the successful creation of the merged VCF file.

In [13]:
docker run --rm -v $tmpdir:/data ghcr.io/ecrrm/genomics-commons bcftools view /data/merged.vcf --no-header | head

chr1	727034	.	C	T	.	PASS	AC=1;AN=2;KFF=0;KFP=0;KM=5.71;MTD=bwa_freebayes,bwa_gatk,bwa_platypus;SF=0	GT	1|0	.
chr1	727242	.	G	A	.	PASS	AC=1;AN=2;KFF=0;KFP=0;KM=9.86;MTD=bwa_freebayes,bwa_gatk,bwa_platypus,isaac_strelka;SF=0	GT	1|0	.
chr1	727477	.	G	A	.	PASS	AC=3;AN=4;KFF=0;KFP=0;KM=6.89;MTD=bwa_freebayes,bwa_gatk,bwa_platypus;SF=0,1	GT	0|1	1|1
chr1	727717	.	G	C	.	PASS	AC=3;AN=4;KFF=0;KFP=0;KM=11.2;MTD=bwa_freebayes,bwa_gatk,bwa_platypus;SF=0,1	GT	0|1	1|1
chr1	729886	.	T	C	.	PASS	AC=1;AN=2;KFF=0;KFP=0;KM=8.29;MTD=bwa_freebayes,bwa_gatk,bwa_platypus;SF=0	GT	1|0	.
chr1	736852	.	C	T	.	PASS	AC=1;AN=2;KFF=0;KFP=0;KM=12.1;MTD=bwa_freebayes,bwa_gatk,bwa_platypus;SF=0	GT	1|0	.
chr1	740738	.	C	T	.	PASS	AC=1;AN=2;KFF=0;KFP=0;KM=9.5;MTD=bwa_freebayes,bwa_gatk,bwa_platypus;SF=1	GT	.	1|0
chr1	744224	.	C	A	.	PASS	AC=4;AN=4;KFF=0;KFP=0;KM=8.19;MTD=bwa_freebayes,bwa_gatk,bwa_platypus;SF=0,1	GT	1|1	1|1
chr1	758351	.	A	G	.	PASS	AC=1;AN=2;KFF=0;KFP=0;KM=11.5;MTD=bwa_freebayes,bwa_gatk,bwa_platypus;SF=0	GT	

The displayed lines provide a glimpse into the merged VCF file's content. Each line represents a genetic variant, and the columns provide details such as the chromosome (`chr`), position, reference allele, alternative allele, and annotations. These details are essential for interpreting the genetic information contained in the VCF file.

## Data Analysis

Before diving into data analysis, we'll convert the merged VCF file into PLINK binary format. PLINK is a widely-used bioinformatics toolset designed for whole-genome association and population-based linkage analysis. By converting our data into PLINK's binary format, we can leverage PLINK's suite of utilities for various genomic analyses.

In [14]:
docker run --rm -v $tmpdir:/data ghcr.io/ecrrm/genomics-commons plink --vcf /data/merged.vcf --make-bed --out /data/plink_data

PLINK v1.90b7 64-bit (16 Jan 2023)             www.cog-genomics.org/plink/1.9/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /data/plink_data.log.
Options in effect:
  --make-bed
  --out /data/plink_data
  --vcf /data/merged.vcf

515347 MB RAM detected; reserving 257673 MB for main workspace.
--vcf: /data/plink_data-temporary.bed + /data/plink_data-temporary.bim +
/data/plink_data-temporary.fam written.
5562641 variants loaded from .bim file.
2 people (0 males, 0 females, 2 ambiguous) loaded from .fam.
Ambiguous sex IDs written to /data/plink_data.nosex .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.741072.
5562641 variants

The output confirms the successful conversion of the merged VCF file to PLINK binary format. From the logs, we can gather that 5,562,641 variants were processed and 2 samples were included in the dataset. This binary format is now ready for further filtering and using PLINK.

In genomics analyses, it's essential to filter out variants with high missing genotyping rates. Such variants can introduce noise and reduce the reliability of downstream analyses. Here, we'll remove variants with a missing genotyping rate greater than 5% to ensure data quality.

The PLINK log provides detailed feedback on the filtering process based on the missing genotyping rate. Post-filtering, a subset of variants is retained, ensuring higher data quality for subsequent analyses.

In [15]:
docker run --rm -v $tmpdir:/data ghcr.io/ecrrm/genomics-commons plink --bfile /data/plink_data --geno 0.05 --make-bed --out /data/plink_data_geno_filtered

PLINK v1.90b7 64-bit (16 Jan 2023)             www.cog-genomics.org/plink/1.9/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /data/plink_data_geno_filtered.log.
Options in effect:
  --bfile /data/plink_data
  --geno 0.05
  --make-bed
  --out /data/plink_data_geno_filtered

515347 MB RAM detected; reserving 257673 MB for main workspace.
5562641 variants loaded from .bim file.
2 people (0 males, 0 females, 2 ambiguous) loaded from .fam.
Ambiguous sex IDs written to /data/plink_data_geno_filtered.nosex .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.741072.
2866935 variants removed due to missing genotype data (--geno).
2695706

From the log, we can see that 5,562,641 variants have been loaded from the .bim file. Then 2,866,935 variants have been removed due to the missing genotype data criteria (based on the `--geno 0.05` option). Thus, after filtering, 2,695,706 variants pass the criteria and remain in the dataset.

Another crucial filtering step in genomics is based on the minor allele frequency (MAF). Variants with a very low MAF can be rare or even erroneous, and their inclusion might affect the robustness of statistical analyses. To maintain data quality, we'll remove variants with a MAF less than 1%.

In [16]:
docker run --rm -v $tmpdir:/data ghcr.io/ecrrm/genomics-commons plink --bfile /data/plink_data_geno_filtered --maf 0.01 --make-bed --out /data/plink_data_maf_filtered

PLINK v1.90b7 64-bit (16 Jan 2023)             www.cog-genomics.org/plink/1.9/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /data/plink_data_maf_filtered.log.
Options in effect:
  --bfile /data/plink_data_geno_filtered
  --maf 0.01
  --make-bed
  --out /data/plink_data_maf_filtered

515347 MB RAM detected; reserving 257673 MB for main workspace.
2695706 variants loaded from .bim file.
2 people (0 males, 0 females, 2 ambiguous) loaded from .fam.
Ambiguous sex IDs written to /data/plink_data_maf_filtered.nosex .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is exactly 1.
944378 variants removed due to minor allele threshold(s)
(--

The output from PLINK provides details on the filtering process based on minor allele frequency. From the log, we can deduce that 944,378 variants were removed due to the specified MAF threshold. After this filtering step, 1,751,328 variants remain, ensuring the dataset is of high quality for the next stages of analysis.

In [17]:
docker run --rm -v $tmpdir:/data ghcr.io/ecrrm/genomics-commons plink --bfile /data/plink_data_maf_filtered --genome --out /data/plink_ibd_output

PLINK v1.90b7 64-bit (16 Jan 2023)             www.cog-genomics.org/plink/1.9/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /data/plink_ibd_output.log.
Options in effect:
  --bfile /data/plink_data_maf_filtered
  --genome
  --out /data/plink_ibd_output

515347 MB RAM detected; reserving 257673 MB for main workspace.
1751328 variants loaded from .bim file.
2 people (0 males, 0 females, 2 ambiguous) loaded from .fam.
Ambiguous sex IDs written to /data/plink_ibd_output.nosex .
Using up to 95 threads (change this with --threads).
Before main variant filters, 2 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is exactly 1.
1751328 variants and 2 people pass filters and QC.
Note: No phenotypes present.
Excluding 27110 vari

The PLINK log provides feedback on the identity-by-descent (IBD) analysis process. From the output, we can deduce that 1,751,328 variants were considered for the IBD analysis for 2 samples. The generated output file (`plink_ibd_output.genome`) contains detailed IBD information for the samples, which can be further analyzed to understand relatedness and shared DNA segments between them.

### Explanation of IBD Analysis Output:

The IBD (Identity-By-Descent) analysis calculates the proportion of the genome that two individuals share due to recent common ancestry. This is represented by the PI_HAT value. In the context of PLINK:

- `Z0`: Proportion of genome shared by two individuals where neither has the segment inherited from a recent common ancestor.
- `Z1`: Proportion of genome shared where one of the individuals has the segment from a recent common ancestor.
- `Z2`: Proportion of genome shared where both individuals have the segment from the recent common ancestor.
- `PI_HAT`: Estimated proportion of genome shared IBD (Z1/2 + Z2).

| Relationship                    | Expected `PI_HAT` Value |
|---------------------------------|-------------------------|
| Unrelated individuals           | 0                       |
| Parent-child                    | 0.5                     |
| Full siblings                   | ~0.5                    |
| Half siblings                   | 0.25                    |
| Grandparent-grandchild          | 0.25                    |
| Uncle-niece or Aunt-nephew      | 0.25                    |
| Double first cousins            | ~0.25                   |
| First cousins                   | 0.125                   |
| First cousins once removed      | 0.0625                  |
| Second cousins                  | 0.03125                 |
| Second cousins once removed     | ~0.015625               |
| Third cousins                   | 0.0078125               |

Therefore, given the known relationship between NA12877 and NA12878 (child and mother), we would expect a `PI_HAT` value around 0.5 from the IBD analysis.

In [18]:
cat $tmpdir/plink_ibd_output.genome

     FID1     IID1     FID2     IID2 RT    EZ      Z0      Z1      Z2  PI_HAT PHE       DST     PPC   RATIO
  NA12877  NA12877  NA12878  NA12878 UN    NA  0.0000  0.8980  0.1020  0.5510  -1  0.739634  1.0000      NA


**From the output**: NA12877 and NA12878 have a PI_HAT value of 0.5510, suggesting they share approximately 55.10% of their genomes due to recent common ancestry. This value is consistent with a parent-offspring relationship, which is expected as NA12878 is the mother of NA12877.

## Conclusion
In this tutorial, we used the Genomics Commons Docker Image. Starting with the download of genomic sample data, we merged VCF files, providing a consolidated view of genetic variants from different samples. The subsequent conversion to PLINK binary format and the filtering steps ensured the data's quality for our primary analysis - the IBD analysis. Through this, we discerned the genetic relatedness between samples NA12877 and NA12878, validating known relationships.

## References
- [PLINK Documentation](https://www.cog-genomics.org/plink/1.9/)
- [VCF Format Specification](https://samtools.github.io/hts-specs/VCFv4.2.pdf)
- [Docker Documentation](https://docs.docker.com/)